In [1]:
import pandas as pd
import sklearn as sfs
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
from model_handler import ModelHandler
from feature_selection import FeatureSelectionAndGeneration
handler = ModelHandler()
dataset = handler.dataset
train_set = dataset[handler.train_mask]

The dataset includes different risks that need a prediction. Every risk is considered as a different target of labels, namely a response variable.

The aim is to build a model able to predict each risk in the most accurate way possible. However, the learning process is different for each of them, meaning that the minimum set of variables that best explain the largest amount of variance in the dataset is unique for every risk. As a consequence, the following pipeline will be executed as much time as the number of risks in order to return as more precise predictions as possible. 

# Dataset splitting

The first step consists in splitting the dataset into training and test sets. The first will be used during the feature selection part, which is implemented using a boosted logistic regression model. This is a supervised learning approach, thus labels are needed for the regression to be carried out. In this dataset risks are assigned to only some of the cities, therefore it's wise to select as training set all the entries containing values for the given risk. All the rest will be referred to as test set, used for the classification task, since those cities will be the ones needing a prediction.

# Feature selection

When there is a highly non-linear and complex relationship between the predictors and the labels decision trees are preferable. The dataset has many different predictors and we don't know whether this relationship is linear or not.

The most robust approach among the ensemble method is `Boosting`. It allows to aggregate many decision trees, differently from `Random Forest`, and grow them sequentially, instead of using boostrap sampling like in `Bagging`. 

The procedure consists in fitting small trees to the residuals in order to slowly improve the prediction error. Generally, model that learn slowly tend to perform better. A pitfall of Boosting, however, is that it relies very much on its tuning parameters. Hence, it's important to undergo `Cross Validation` in order to select the combination returning the highest accuracy, for every target. 
For this purpose we decided to use 10-fold cross validation in such a way to speed up the tuning process, which is already slow given the amount of parameters that need to be optimized.

In [2]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
import shutil
import os
memory_dir = '.pipeline_cache.tmp'

XgBoost has as default objective function `reg:squarederror`, which corresponds to a linear regression with mean-squared error as loss function.

In [3]:
from bayes_opt import BayesianOptimization
if os.path.isdir(memory_dir):
    shutil.rmtree(memory_dir)

def init_model(**model_params):
    return Pipeline([('generation_and_selection', FeatureSelectionAndGeneration(feats_num=200)), ('regressor', xgb.XGBRegressor(**model_params))],memory=memory_dir)
    

In [4]:
from sklearn.model_selection import cross_val_score
from data.labeled.preprocessed import RISKS_MAPPING
from classification import RANDOM_SEED
optimal_params = {}
CONSTANTS = {'subsample': 0.8, 'objective':"reg:squarederror", "random_state": RANDOM_SEED,'subsample':1}
for (risk, total_set, [train_set, valid_set]) in handler.get_total_train_val_set_per_risk():
    print(f"\n\n**Risk: {RISKS_MAPPING[risk]}**\n")
    print(f"Annotated Samples Size: {total_set.shape[0]}")
    print(f"To be used for parameters estimation: {train_set.shape[0]}\n")
    def xgb_evaluate(max_depth, 
                     gamma, 
                     alpha,
                     colsample_bytree, n_estimators, learning_rate):
        params = {'max_depth': int(max_depth),
                  'subsample': 0.8,
                  'alpha': alpha,
                  'gamma': gamma,
                  'colsample_bytree': colsample_bytree,
                   'n_estimators': int(n_estimators),
                 'learning_rate':learning_rate}
        params.update(CONSTANTS)

        model = init_model(**params)
        train_tuple = (train_set[handler.feat_names], train_set[risk])
        reg_cv = model.fit(*train_tuple)
        cv_result = np.mean(cross_val_score(model, *train_tuple, cv=3,scoring='neg_mean_squared_error'))
        return cv_result
    xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (1, 7), 
                                                 'alpha': (0,20),
                                                 'gamma': (0, 1),
                                                 'colsample_bytree': (0.3, 0.9),
                                                 "n_estimators":[200,1000],
                                                 "learning_rate":[0.1,0.5]
                                                }
                                  
                                 )
    
    # Use the expected improvement acquisition function to handle negative numbers
    # Optimally needs quite a few more initiation points and number of iterations
    xgb_bo.maximize(init_points=10, n_iter=10)
    params = xgb_bo.max['params']
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params.update(CONSTANTS)
    optimal_params[risk] = params



**Risk: Higher water prices**

Annotated Samples Size: 87
To be used for parameters estimation: 60

|   iter    |  target   |   alpha   | colsam... |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------------------------------
|  1        | -0.8822   |  9.765    |  0.8739   |  0.8002   |  0.4647   |  3.724    |  404.7    |
|  2        | -0.8573   |  14.01    |  0.4157   |  0.3896   |  0.439    |  3.458    |  841.8    |
|  3        | -0.85     |  18.6     |  0.8356   |  0.8061   |  0.2754   |  6.371    |  639.0    |
|  4        | -1.025    |  1.612    |  0.4721   |  0.7719   |  0.3364   |  1.057    |  295.0    |
|  5        | -1.35     |  0.009164 |  0.6718   |  0.3831   |  0.1214   |  4.294    |  477.8    |
|  6        | -0.8745   |  10.79    |  0.3788   |  0.3671   |  0.301    |  4.162    |  361.8    |
|  7        | -0.9235   |  4.812    |  0.367    |  0.8025   |  0.4357   |  2.485    |  498.1    |
|  8        | -0

|  5        | -0.3917   |  8.059    |  0.6758   |  0.496    |  0.3194   |  4.238    |  906.0    |
|  6        | -0.3958   |  12.2     |  0.7957   |  0.7235   |  0.3622   |  6.863    |  285.0    |
|  7        | -0.3805   |  6.472    |  0.6031   |  0.7971   |  0.4675   |  4.928    |  755.6    |
|  8        | -0.3831   |  0.9752   |  0.6736   |  0.6508   |  0.1252   |  5.412    |  768.5    |
|  9        | -0.379    |  12.26    |  0.3881   |  0.7877   |  0.1248   |  6.233    |  955.3    |
|  10       | -0.4726   |  1.209    |  0.8003   |  0.05304  |  0.1982   |  6.629    |  443.3    |
|  11       | -0.3826   |  13.86    |  0.5845   |  0.7992   |  0.4093   |  2.946    |  253.5    |
|  12       | -0.5498   |  0.0      |  0.3      |  0.0      |  0.5      |  7.0      |  200.0    |
|  13       | -0.4559   |  0.0      |  0.3      |  0.0      |  0.1      |  1.0      |  691.4    |
|  14       | -0.3792   |  12.0     |  0.487    |  0.7245   |  0.1724   |  4.741    |  955.7    |
|  15       | -0.403

|  11       | -1.261    |  13.76    |  0.3578   |  0.1798   |  0.4457   |  3.626    |  993.4    |
|  12       | -1.244    |  15.95    |  0.4207   |  0.4039   |  0.4174   |  5.11     |  981.2    |
|  13       | -1.222    |  16.33    |  0.5447   |  0.7159   |  0.4246   |  6.845    |  589.0    |
|  14       | -1.353    |  19.94    |  0.551    |  0.9992   |  0.3305   |  2.007    |  581.5    |
|  15       | -1.209    |  11.74    |  0.8434   |  0.3206   |  0.2305   |  5.962    |  590.8    |
|  16       | -1.26     |  17.23    |  0.4266   |  0.3869   |  0.2747   |  4.042    |  598.8    |
|  17       | -1.279    |  4.26     |  0.6554   |  0.6014   |  0.3875   |  6.456    |  592.9    |
|  18       | -1.217    |  9.776    |  0.9      |  1.0      |  0.5      |  1.0      |  460.7    |
|  19       | -1.419    |  3.613    |  0.3868   |  0.2436   |  0.1393   |  5.808    |  463.5    |
|  20       | -1.261    |  13.12    |  0.3      |  1.0      |  0.5      |  2.016    |  589.3    |


**Risk: Regulatory

|  17       | -0.5309   |  0.783    |  0.463    |  0.6296   |  0.3681   |  4.155    |  389.3    |
|  18       | -0.5579   |  0.7492   |  0.556    |  0.9379   |  0.1519   |  2.707    |  390.5    |
|  19       | -0.5184   |  1.476    |  0.6585   |  0.5587   |  0.48     |  4.013    |  391.6    |
|  20       | -0.5343   |  1.425    |  0.3264   |  0.9839   |  0.3542   |  4.404    |  389.1    |


In [5]:
from data.model import MODEL_BEST_PARAMS_PATH
pd.DataFrame(optimal_params).to_csv(MODEL_BEST_PARAMS_PATH)